In [2]:
from IPython.display import display, clear_output
import ipywidgets as widgets

from arcgis.features import SpatialDataFrame
from arcgis.geoenrichment import enrich
from arcgis.raster import ImageryLayer
from arcgis.geometry import Geometry
from arcgis.gis import GIS

gis = GIS("")
img_svc ='https://landscape7.arcgis.com/arcgis/rest/services/World_Population_Density_Estimate_2016/ImageServer/'
img_lyr = ImageryLayer(img_svc, gis=gis)

In [3]:
%%html
<style>
.intro {
    padding: 10px; 
    color: #202020;
    font-family: 'Helvetica'
}
.map {
    border: solid;
    height: 450;
}
</style>

In [4]:
# run_btn on_click
def running(button_object):
    viz_map.clear_graphics()
    the_geom = Geometry(viz_map.extent)
    enriched = SpatialDataFrame(enrich([the_geom], gis=gis))
    
    if 'TOTPOP' not in list(enriched):
        enriched_pop = -1
    else:
        enriched_pop = enriched.TOTPOP[0]
    
    print('the Geom: {}'.format(the_geom))
    print('Sample Count: {}'.format(img_lyr.properties.maxImageHeight))
    
    samples = img_lyr.get_samples(the_geom)
    sample_sum = sum([float(sample['value']) for sample in samples])
    print('GeoEnrichment Population Results: {}'.format(enriched_pop))

    title = 'GeoEnriched Cell'
    for feature in enriched.to_featureset():
        # Expand Attributes With New Lines
        content = "{}".format(
            '<br/>'.join(['%s: %s' % (key.upper(), value) for (key, value) in feature.attributes.items()])
        )
        # Add Cell to Map
        viz_map.draw(feature.geometry, popup={'title': title, 'content': content})

In [5]:
# Create & Display Map
viz_map = gis.map('St Louis')
viz_map.add_layer(img_lyr)
display(viz_map)

# Create Submit Button & Set on_click
run_btn = widgets.Button(
    description='Fetch Extent',
    button_style='success',
    tooltip='Collect GeoEnrichment for Extent',
    layout=widgets.Layout(justify_content='center', margin='0px 0px 0px 10px')
)
run_btn.on_click(running)

# Handle Widget Layout
params = widgets.HBox(
    [run_btn], 
    layout=widgets.Layout(justify_content='center', margin='10px')
)
display(params)

Exception: Token Required
(Error Code: 499)